# Subsistema de Privacidad y Seguridad basado en Redis Cluster

Este notebook implementa el Subsistema de Privacidad y Seguridad de Duolingo utilizando **Redis Cluster** como motor principal de almacenamiento para datos sensibles, auditoría, control de acceso y operaciones efímeras.

A diferencia de una instancia única de Redis, aquí se usa un **cluster con múltiples nodos maestro y réplica**, lo que permite:
- Alta disponibilidad mediante replicación y failover automático.
- Distribución de claves por slots (sharding) entre varios nodos.
- Persistencia configurable por nodo usando **RDB** (snapshots) y/o **AOF** (Append Only File).

Redis se ejecuta localmente mediante Docker Compose. Uno de los nodos del cluster (por ejemplo, `redis-1`) se usa como punto de entrada desde la notebook, y el driver de Redis se encarga de enrutar las operaciones al nodo correcto según la clave.

---

## Objetivo del Notebook

El objetivo es demostrar los principales patrones de acceso del subsistema, cada uno implementado utilizando las primitivas de Redis adecuadas.

### 1. Validación de tokens y control de acceso (RBAC + Tokens)
Incluye:
- Conjuntos (Sets) para representar roles y permisos (RBAC).
- Strings con TTL para tokens de sesión.
- Marcas de revocación de tokens.
- Streams de auditoría para registrar validaciones y accesos.

### 2. Actualización de preferencias y gestión de consentimientos
Incluye:
- Almacenamiento de preferencias en formato JSON.
- Historial de consentimientos mediante listas.
- Registro automático de auditoría ante cada modificación.

### 3. Registro y consumo de eventos de auditoría
Incluye:
- Stream global de auditoría (`audit:events`).
- Stream individual por usuario (`audit:byUser:{userId}`).
- Consultas cronológicas y reversas.

### 4. Operaciones efímeras con expiración automática
Incluye:
- Uso de TTL para asegurar eliminación automática.
- Datos temporales como tokens o claves de recuperación.

### 5. Eliminación y anonimización asincrónica
Incluye:
- Uso de streams como colas de trabajo (`privacy:deleteQueue`).
- Simulación de un proceso worker que elimina o anonimiza datos.
- Registro de auditoría de cada operación ejecutada.

### 6. Métricas de usuarios activos con HyperLogLog
Incluye:
- Uso de **HyperLogLog** para estimar la cantidad de usuarios activos.
- Actualización automática de la métrica al emitir tokens.

---

# Modelo de Datos y Namespaces

El subsistema organiza la información mediante claves jerárquicas con el formato:

`<dominio>:<subdominio>:<identificador>[:atributo]`

Ejemplos de claves utilizadas:

- `sec:role:{roleId}` → conjunto de permisos asociados a un rol.
- `sec:userRoles:{userId}` → conjunto de roles asignados a un usuario.
- `sec:token:{jti}` → token de sesión con TTL.
- `sec:revoked:{jti}` → marca de token revocado.
- `privacy:prefs:{userId}` → preferencias de privacidad del usuario.
- `privacy:consent:{userId}` → historial de consentimientos.
- `audit:events` → stream global de auditoría.
- `audit:byUser:{userId}` → stream por usuario.
- `privacy:deleteQueue` → cola de solicitudes de eliminación o anonimización.
- `sec:active_users_hll` → HyperLogLog con usuarios activos.

Este esquema facilita la separación de responsabilidades, la aplicación de políticas diferenciadas de expiración y la trazabilidad completa de operaciones.

---

# Archivo auxiliar

Para mantener la notebook organizada y modular, toda la lógica del subsistema se implementa en el archivo auxiliar `security_setup.py`.

Este archivo define, entre otros:

- Funciones para crear permisos y asignar roles a usuarios.
- Lógica de RBAC y validación de permisos.
- Emisión, validación y revocación de tokens con TTL.
- Actualización de métricas de usuarios activos mediante HyperLogLog.
- Gestión de preferencias de privacidad y consentimientos.
- Registro y consulta de eventos mediante Redis Streams.
- Encolado y procesamiento de solicitudes de eliminación y anonimización.
- Conexión al cluster de Redis definido en Docker Compose.

La notebook importa estas funciones y se concentra exclusivamente en demostrar los patrones de acceso y la persistencia de los datos según la configuración del cluster de Redis.

In [72]:
%pip install redis

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


# Conexión con Redis Cluster

En esta celda se establece la conexión contra el cluster de Redis definido en `docker-compose.yml`.

- Si se ejecuta la notebook dentro del contenedor de Jupyter del `docker-compose`, se puede usar como host uno de los nodos del cluster, por ejemplo `redis-1`.
- Si se ejecuta la notebook de forma local en VS Code, se puede usar `localhost` y el puerto mapeado (por ejemplo, `7001`).

La función `connect` definida en `security_setup.py` devuelve un cliente configurado para trabajar con Redis Cluster.

In [73]:
from security_setup import *

r = connect_cluster([
    {"host": "localhost", "port": 7001},
    {"host": "localhost", "port": 7002},
    {"host": "localhost", "port": 7003},
])
print("Conectado al cluster:", r.ping())

Conectado al cluster: True


In [79]:
print("=== Topología del Redis Cluster ===")

nodes = r.cluster_nodes()

for addr, info in nodes.items():
    print(f"\nAddress: {addr}")
    print(f"  Node ID: {info.get('node_id')}")
    print(f"  Flags: {info.get('flags')}")
    
    role = "master" if "master" in info.get("flags", []) else "replica"
    print(f"  Role: {role}")
    
    master_id = info.get("master_id")
    if role == "replica":
        print(f"  Replica of: {master_id}")

    slots = info.get("slots", [])
    if role == "master":
        print(f"  Slots: {slots}")


=== Topología del Redis Cluster ===

Address: 192.168.1.3:7006
  Node ID: 980ec3cb207ca0db874628969c3ebc2443433de8
  Flags: slave
  Role: replica
  Replica of: 0a1f57c9154d6fc0166b126304c77f6badf6a5f2

Address: 192.168.1.3:7002
  Node ID: 3244903afae215b90e741161696df962c9709373
  Flags: master
  Role: master
  Slots: [['5461', '10922']]

Address: 192.168.1.3:7005
  Node ID: 8b5533adbe4f97deb5a298ae84d7d1bcec2d43d6
  Flags: slave
  Role: replica
  Replica of: 3244903afae215b90e741161696df962c9709373

Address: 192.168.1.3:7004
  Node ID: c4e0da4f678b63c670553c13f9c77c12ffdc7ed5
  Flags: slave
  Role: replica
  Replica of: 4a03536ffb4737a457b0f750819e395ae44e12d2

Address: 192.168.1.3:7003
  Node ID: 0a1f57c9154d6fc0166b126304c77f6badf6a5f2
  Flags: master
  Role: master
  Slots: [['10923', '16383']]

Address: 192.168.1.3:7001
  Node ID: 4a03536ffb4737a457b0f750819e395ae44e12d2
  Flags: myself,master
  Role: master
  Slots: [['0', '5460']]


## Patrón 1: Validación de Tokens y Control de Acceso (RBAC) – Escritura

En esta sección se configura el control de acceso basado en roles (RBAC) y se emite un token de sesión:

- Se asignan permisos al rol `student`.
- Se asignan permisos al rol `moderator`.
- Se asignan roles a usuarios (`user:1`, `user:2`).
- Se emite un token de sesión con TTL para `user:1`.

La emisión del token registra eventos de auditoría y actualiza métricas de usuarios activos mediante HyperLogLog.

In [80]:
from security_setup import *
# Configuración de roles y permisos
add_permission_to_role(r, "student", "user:view_profile")
add_permission_to_role(r, "moderator", "audit:view")

# Asignación de roles a usuarios
assign_role_to_user(r, "user:1", "student")
assign_role_to_user(r, "user:2", "moderator")

# Emisión de un token de sesión con TTL para user:1
jti = issue_token(r, "user:1", ttl_seconds=120)
print("Token emitido para user:1:", jti)

# Guardamos el último JTI en una variable global para usarlo en celdas posteriores
last_jti = jti

Token emitido para user:1: 7b5186f5-608c-4442-a925-6892f85762a0


## Lectura y Verificación del Patrón RBAC + Tokens

En esta sección se verifica que la información creada en la celda anterior está disponible y consistente:

- Se listan los roles de `user:1`.
- Se listan los permisos asociados al rol `student`.
- Se evalúa si `user:1` posee permisos específicos.
- Se valida el token emitido anteriormente (`last_jti`).

Esta lectura permite comprobar también el efecto de la configuración de persistencia del cluster (RDB/AOF) entre reinicios del servicio.

In [81]:
print("Roles de user:1:", get_user_roles(r, "user:1"))
print("Permisos del rol student:", get_role_permissions(r, "student"))

print("user:1 puede ver perfil? =>", user_has_permission(r, "user:1", "user:view_profile"))
print("user:1 puede ver auditoría? =>", user_has_permission(r, "user:1", "audit:view"))

print("Validación del token last_jti:")
print(validate_token(r, last_jti))

Roles de user:1: {'student'}
Permisos del rol student: {'user:view_profile'}
user:1 puede ver perfil? => True
user:1 puede ver auditoría? => False
Validación del token last_jti:
{'user_id': 'user:1', 'issued_at': '2025-12-04T21:52:41.155553', 'expires_at': '2025-12-04T21:54:41.155553', 'scope': []}


## Patrón 2: Actualización de Preferencias y Gestión de Consentimientos – Escritura

En esta sección se crean y actualizan estructuras relacionadas con privacidad:

- Preferencias de privacidad en `privacy:prefs:{userId}`.
- Historial de consentimientos en `privacy:consent:{userId}`.

        Cada operación genera un evento de auditoría en los streams correspondientes.

In [82]:
prefs_u1 = {
    "profile_visibility": "friends_only",
    "share_learning_stats": True,
    "share_location": False,
}

set_privacy_prefs(r, "user:1", prefs_u1)

add_consent_entry(r, "user:1", "terms_and_conditions", True)
add_consent_entry(r, "user:1", "marketing_emails", False)

print("Preferencias y consentimientos creados para user:1")

Preferencias y consentimientos creados para user:1


## Lectura de Preferencias y Consentimientos

En esta sección se leen las estructuras creadas anteriormente para verificar que los datos persisten correctamente:

- Recuperación de las preferencias de `user:1`.
- Listado del historial de consentimientos asociados a `user:1`.

In [83]:
print("Preferencias persistidas para user:1:")
print(get_privacy_prefs(r, "user:1"))

print("\nConsentimientos guardados para user:1:")
for raw in r.lrange(key_privacy_consent("user:1"), 0, -1):
    print("-", json.loads(raw))

Preferencias persistidas para user:1:
{'profile_visibility': 'friends_only', 'share_learning_stats': True, 'share_location': False}

Consentimientos guardados para user:1:
- {'timestamp': '2025-12-04T21:53:18.145032', 'type': 'terms_and_conditions', 'granted': True}
- {'timestamp': '2025-12-04T21:53:18.147102', 'type': 'marketing_emails', 'granted': False}


## Patrón 3: Registro de Auditoría con Redis Streams – Escritura

Aquí se generan algunos eventos de auditoría adicionales para demostrar:

- Registro en el stream global `audit:events`.
- Registro en el stream por usuario `audit:byUser:{userId}`.

Los eventos representan acciones típicas como visualización y actualización de privacidad, o validaciones de tokens.

In [85]:
add_audit_event(r, "user:1", "privacy_view", "success", {"section": "profile"})
add_audit_event(r, "user:2", "privacy_update", "success", {"field": "visibility"})
add_audit_event(r, "user:1", "token_validation", "failure", {"reason": "expired"})

print("Eventos de auditoría adicionales generados.")

Eventos de auditoría adicionales generados.


## Lectura de Auditoría (Global y Por Usuario)

En esta sección se consultan los streams de auditoría:

- Se obtienen los últimos N eventos del stream global `audit:events`.
- Se obtienen los últimos eventos asociados al usuario `user:1`.

Esto demuestra cómo Redis Streams permiten trazabilidad cronológica y segmentada por usuario.

In [86]:
print("Últimos eventos globales:")
for event_id, fields in get_last_audit_events(r, 10):
    print(event_id, "=>", fields)

print("\nÚltimos eventos de user:1:")
for event_id, fields in get_last_audit_events_by_user(r, "user:1", 10):
    print(event_id, "=>", fields)

Últimos eventos globales:
1764885278118-0 => {'user_id': 'user:1', 'action': 'token_validation', 'result': 'failure', 'timestamp': '2025-12-04T21:54:38.148614', 'metadata': '{"reason": "expired"}'}
1764885278116-0 => {'user_id': 'user:2', 'action': 'privacy_update', 'result': 'success', 'timestamp': '2025-12-04T21:54:38.147577', 'metadata': '{"field": "visibility"}'}
1764885278115-0 => {'user_id': 'user:1', 'action': 'privacy_view', 'result': 'success', 'timestamp': '2025-12-04T21:54:38.145444', 'metadata': '{"section": "profile"}'}
1764885198238-0 => {'user_id': 'user:1', 'action': 'privacy_consent_update', 'result': 'success', 'timestamp': '2025-12-04T21:53:18.147617', 'metadata': '{"timestamp": "2025-12-04T21:53:18.147102", "type": "marketing_emails", "granted": false}'}
1764885198237-0 => {'user_id': 'user:1', 'action': 'privacy_consent_update', 'result': 'success', 'timestamp': '2025-12-04T21:53:18.146065', 'metadata': '{"timestamp": "2025-12-04T21:53:18.145032", "type": "terms_an

## Patrón 4: Claves Efímeras con TTL – Escritura

En esta sección se emite un token de sesión con un TTL muy corto (por ejemplo, 10 segundos) para demostrar:

- Cómo Redis elimina automáticamente la clave una vez expirado el TTL.
- Cómo la validación del token pasa de exitosa a fallida una vez vencido su tiempo de vida.

In [87]:
jti_ttl = issue_token(r, "user:3", ttl_seconds=10)
print("Token efímero emitido para user:3:", jti_ttl)

Token efímero emitido para user:3: 2e1e8682-2e7a-4fff-b026-a6e9cb660ad1


## Verificación de Expiración de Token

En esta sección se verifica el estado del token efímero:

- Se consulta el TTL restante de la clave.
- Se verifica si el token aún existe.
- Se valida el token para observar el resultado antes o después de su expiración.

In [96]:
print("TTL restante del token efímero:", r.ttl(key_token(jti_ttl)))
print("Token efímero existe?", r.exists(key_token(jti_ttl)))
print("Validación del token efímero:", validate_token(r, jti_ttl))

TTL restante del token efímero: 4
Token efímero existe? 1
Validación del token efímero: {'user_id': 'user:3', 'issued_at': '2025-12-04T21:54:43.974197', 'expires_at': '2025-12-04T21:54:53.974197', 'scope': []}


## Patrón 5: Anonimización Asíncrona – Escritura

En esta sección se encola una solicitud de eliminación/anonimización de datos en `privacy:deleteQueue`.

- El encolado representa la acción del usuario solicitando ejercer su derecho al olvido.
- El procesamiento real se delega a un worker asíncrono (simulado en este notebook) que se ejecuta en otra celda.

Esto desacopla la solicitud del usuario del procesamiento intensivo de borrado/anonimización, evitando impactos en la latencia de la API principal.

In [97]:
enqueue_delete_request(r, "user:1", "right_to_be_forgotten")
print("Solicitud de anonimización encolada para user:1.")

Solicitud de anonimización encolada para user:1.


## Procesamiento de Solicitudes de Eliminación

En esta sección se simula el worker que procesa la cola `privacy:deleteQueue`:

- Se leen las solicitudes pendientes.
- Se eliminan roles, preferencias y consentimientos asociados al usuario.
- Se registran los resultados en el stream de auditoría.

Luego se verifica el estado de los datos de `user:1` para confirmar que fueron borrados.

In [98]:
processed = process_delete_requests(r)
print("Solicitudes procesadas:", processed)

print("\nEstado de user:1 luego de la anonimización:")
print("Roles:", get_user_roles(r, "user:1"))
print("Preferencias:", get_privacy_prefs(r, "user:1"))
print("Consentimientos:", r.lrange(key_privacy_consent("user:1"), 0, -1))

Solicitudes procesadas: [('1764885295224-0', {'user_id': 'user:1', 'reason': 'right_to_be_forgotten', 'requested_at': '2025-12-04T21:54:55.280230'})]

Estado de user:1 luego de la anonimización:
Roles: set()
Preferencias: None
Consentimientos: []


## Patrón 6: Métricas de Usuarios Activos con HyperLogLog – Escritura

En este subsistema se utiliza **HyperLogLog** para estimar la cantidad de usuarios activos sin almacenar cada identificador de forma explícita.

La lógica de emisión de tokens (`issue_token`) actualiza automáticamente una estructura HyperLogLog (por ejemplo, `sec:active_users_hll`) cada vez que se genera un token.

En esta celda se emiten tokens para varios usuarios a modo de ejemplo, lo que aumenta el conjunto de usuarios considerados como activos.

In [99]:
for uid in ["user:1", "user:2", "user:3", "user:4", "user:5"]:
    issue_token(r, uid, ttl_seconds=300)

print("Tokens emitidos para usuarios user:1 a user:5.")
print("La métrica HyperLogLog diaria se actualiza en cada issue_token().")

Tokens emitidos para usuarios user:1 a user:5.
La métrica HyperLogLog diaria se actualiza en cada issue_token().


## Lectura de Métricas de Usuarios Activos (HyperLogLog)

En esta sección se consulta la cantidad aproximada de usuarios activos usando la estructura HyperLogLog:

- Se lee el valor aproximado mediante `PFCOUNT` sobre la clave `sec:active_users_hll`.
- Este patrón ejemplifica cómo obtener métricas agregadas sin almacenar listas completas de identificadores de usuario.


In [100]:
from datetime import datetime, timezone 

date_str = datetime.now(timezone.utc).strftime("%Y-%m-%d")
key = key_active_users(date_str)

estimated_active = r.pfcount(key)
print(f"Usuarios activos aproximados para {date_str}: {estimated_active}")


Usuarios activos aproximados para 2025-12-04: 5


## Conclusiones y relación con los requerimientos no funcionales

A partir de los ejemplos anteriores se ve cómo Redis Cluster cumple los puntos de la letra:

1. **Baja latencia y alto rendimiento**  
   - Todas las operaciones clave-valor y de sets se realizan en memoria.  
   - Validación de tokens (`GET`, `EXISTS`) y verificación de permisos (`SISMEMBER`) son O(1).

2. **Alta disponibilidad y tolerancia a fallos**  
   - El uso de Redis Cluster con nodos maestro y réplica permite que, ante la caída de un nodo, otro tome su lugar.  
   - El cluster bus y el mecanismo de failover automático colaboran para mantener los mecanismos de autenticación y validación siempre disponibles.

3. **Escalabilidad horizontal**  
   - El sharding automático distribuye las claves entre varios nodos, permitiendo escalar lecturas y escrituras añadiendo más nodos al cluster.

4. **Manejo de datos efímeros y expiración automática**  
   - Tokens de sesión (`sec:token:{jti}`) se crean con TTL.  
   - Flags de revocación (`sec:revoked:{jti}`) utilizan también TTL para mantener la coherencia.

5. **Seguridad y cifrado (a nivel conceptual)**  
   - En un entorno productivo, Redis puede configurarse con TLS, ACLs por usuario y autenticación obligatoria.  
   - La notebook se focaliza en el modelo de datos y patrones de acceso, pero el cluster podría complementarse con estas medidas.

6. **Consistencia y durabilidad configurables**  
   - Redis ofrece RDB y AOF para persistir información crítica como los streams de auditoría, manteniendo un equilibrio entre rendimiento y durabilidad.  
   - En un cluster, cada nodo puede ajustarse con la política de persistencia adecuada según el tipo de datos.

7. **Trazabilidad y auditoría continua**  
   - Cada acción relevante (emisión y validación de tokens, cambios de preferencias, solicitudes de borrado) queda registrada en `audit:events` y `audit:byUser:{userId}`.  
   - Redis Streams permiten procesar estos eventos en tiempo real o de forma diferida sin bloquear la operación principal.

8. **Procesos asíncronos y no bloqueantes**  
   - La cola `privacy:deleteQueue` permite gestionar solicitudes de anonimización de forma asíncrona, evitando afectar la experiencia del resto de los usuarios.

9. **Métricas agregadas eficientes (HyperLogLog)**  
   - El uso de HyperLogLog para medir usuarios activos permite obtener métricas globales con muy bajo consumo de memoria.

En conjunto, este subsistema ilustra cómo Redis Cluster puede actuar como **motor central de seguridad, privacidad, auditoría y métricas**, cumpliendo con los principios de confidencialidad, integridad, disponibilidad, trazabilidad y escalabilidad exigidos por la plataforma.